In [1]:
entity_unit_map = {
    'width': [
        ['centimetre', 'centimetres', 'cm', 'cms'],
        ['foot', 'feet', 'ft'],
        ['inch', 'inches', 'in', '"'],
        ['metre', 'meter', 'meters', 'm'],
        ['millimetre', 'millimeter', 'millimeters', 'mm'],
        ['yard', 'yards', 'yd', 'yds']
    ],
    'depth': [
        ['centimetre', 'centimetres', 'cm', 'cms'],
        ['foot', 'feet', 'ft'],
        ['inch', 'inches', 'in'],
        ['metre', 'meter', 'meters', 'm'],
        ['millimetre', 'millimeter', 'millimeters', 'mm'],
        ['yard', 'yards', 'yd', 'yds']
    ],
    'height': [
        ['centimetre', 'centimetres', 'cm', 'cms'],
        ['foot', 'feet', 'ft'],
        ['inch', 'inches', 'in'],
        ['metre', 'meter', 'meters', 'm'],
        ['millimetre', 'millimeter', 'millimeters', 'mm'],
        ['yard', 'yards', 'yd', 'yds']
    ],
    'item_weight': [
        ['gram', 'grams', 'g'],
        ['kilogram', 'kilograms', 'kg', 'kgs'],
        ['microgram', 'micrograms', 'μg', 'mcg'],
        ['milligram', 'milligrams', 'mg'],
        ['ounce', 'ounces', 'oz'],
        ['pound', 'pounds', 'lb', 'lbs'],
        ['ton', 'tons', 't']
    ],
    'maximum_weight_recommendation': [
        ['gram', 'grams', 'g'],
        ['kilogram', 'kilograms', 'kg', 'kgs'],
        ['microgram', 'micrograms', 'μg', 'mcg'],
        ['milligram', 'milligrams', 'mg'],
        ['ounce', 'ounces', 'oz'],
        ['pound', 'pounds', 'lb', 'lbs'],
        ['ton', 'tons', 't']
    ],
    'voltage': [
        ['kilovolt', 'kilovolts', 'kv'],
        ['millivolt', 'millivolts', 'mv'],
        ['volt', 'volts', 'v']
    ],
    'wattage': [
        ['kilowatt', 'kilowatts', 'kw'],
        ['watt', 'watts', 'w']
    ],
    'item_volume': [
        ['centilitre', 'centiliter', 'centiliters', 'cl'],
        ['cubic foot', 'cubic feet', 'cu ft', 'ft³'],
        ['cubic inch', 'cubic inches', 'cu in', 'in³'],
        ['cup', 'cups'],
        ['decilitre', 'deciliter', 'deciliters', 'dl'],
        ['fluid ounce', 'fluid ounces', 'fl oz'],
        ['gallon', 'gallons', 'gal'],
        ['imperial gallon', 'imperial gallons', 'imp gal'],
        ['litre', 'liter', 'liters', 'litres', 'l'],
        ['microlitre', 'microliter', 'microliters', 'microlitres', 'μl', 'mcl'],
        ['millilitre', 'milliliter', 'milliliters', 'millilitres', 'ml'],
        ['pint', 'pints', 'pt'],
        ['quart', 'quarts', 'qt']
    ]
}


In [2]:
import os
import re
from urllib.parse import urlparse
import pandas as pd
from paddleocr import PaddleOCR

# Initialize the PaddleOCR model with the angle classifier
ocr = PaddleOCR(use_angle_cls=True)

# Define relative paths
BASE_PATH = os.getcwd()
IMAGES_FOLDER = os.path.join(BASE_PATH, 'Test_images')
DATASET_FOLDER = os.path.join(BASE_PATH, 'dataset')

# Function to run OCR on an image
def run_OCR(image_name):
    image_path = os.path.join(IMAGES_FOLDER, image_name)
    result = ocr.ocr(image_path)
    return result

def name_parser(image_link):
    parsed_url = urlparse(image_link)
    path = parsed_url.path
    image_name = path.split('/')[-1]
    return image_name

def split_alphanumeric_with_special_chars(string):
    matches = re.findall(r'(\d+\.?\,?\d*)([a-zA-Z]+)?', string)
    numbers_with_units = []
    standalone_numbers = []
    
    for match in matches:
        number, unit = match
        number = number.replace(",", ".")
        if unit:
            numbers_with_units.append((float(number), unit))
        else:
            standalone_numbers.append(float(number))
    
    return numbers_with_units, standalone_numbers

def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def get_center(coordinates):
    return (sum(coord[0] for coord in coordinates) / 4, sum(coord[1] for coord in coordinates) / 4)

def predictor(image_link, category_id, entity_name):
    OCR_result = run_OCR(name_parser(image_link))
    
    numbers = []
    units = []
    
    for line in OCR_result:
        if not line:
            continue
        for word_info in line:
            text, coordinates = word_info[1][0], word_info[0]
            numbers_with_units, standalone_numbers = split_alphanumeric_with_special_chars(text)
            
            for number, unit in numbers_with_units:
                numbers.append((number, get_center(coordinates)))
                if unit:
                    units.append((unit, get_center(coordinates)))
            
            for number in standalone_numbers:
                numbers.append((number, get_center(coordinates)))
            
            # Check for standalone units
            alphabets = re.findall(r'[a-zA-Z]+', text)
            for word in alphabets:
                for unit_list in entity_unit_map[entity_name]:
                    if word.lower() in [u.lower() for u in unit_list]:
                        units.append((unit_list[0], get_center(coordinates)))
                        break
    
    if not numbers:
        return ""
    
    # Find the best number-unit pair based on distance
    best_pair = None
    min_distance = float('inf')
    
    for number, num_coord in numbers:
        for unit, unit_coord in units:
            distance = calculate_distance(num_coord, unit_coord)
            if distance < min_distance:
                min_distance = distance
                best_pair = (number, unit)
    
    if best_pair:
        number, unit = best_pair
        return f"{number} {unit}"
    elif numbers:  # If we have numbers but no units
        return f"{numbers[0][0]}"
    
    return ""

# Main execution function
if __name__ == "__main__":
    # Paths to input and output files
    input_file = os.path.join(DATASET_FOLDER, 'test.csv')
    output_file = os.path.join(DATASET_FOLDER, 'test_out.csv')
    
    # Read the test data
    test = pd.read_csv(input_file)
    
    # Initialize the starting index from the "index" column
    start_index = 0
    
    # If the output file exists, get the last processed index from the "index" column
    if os.path.exists(output_file):
        try:
            last_line = pd.read_csv(output_file).tail(1)
            start_index = last_line['index'].values[0] + 1
        except Exception as e:
            print(f"Error reading the output file: {e}")
            start_index = 0
    
    # Allow custom starting points, overriding the automatic start index if needed
    custom_start_index = 126094  # You can change this value as per your requirement
    start_index = max(start_index, custom_start_index)
    
    # Empty list to store temporary results
    results_list = []
    
    # Iterate through the rows of the test data starting from start_index in the "index" column
    for _, row in test[test['index'] >= start_index].iterrows():
        index = row['index']
        
        # Check if we need to stop after processing 500 rows
        if (index - start_index) % 50 == 0 and index > start_index:
            print(f"Stopping at index {index}.")
            break
        
        prediction = predictor(row['image_link'], row['group_id'], row['entity_name'])
        
        # Append the result to the list
        results_list.append({'index': index, 'prediction': prediction})
        
        # Every 100 iterations, append to CSV and clear list
        if (index + 1) % 50 == 0:
            results_df = pd.DataFrame(results_list)
            results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            results_list.clear()
    
    # After the loop, write any remaining rows in the list
    if results_list:
        results_df = pd.DataFrame(results_list)
        results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
    
    print("Done")



[2024/09/16 00:50:58] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\prath/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\prath/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [ ]:
# import pyautogui
# import time

# while True:
#     x,y = pyautogui.position()
#     print(f"{x},{y}")
#     time.sleep(1)

In [ ]:
import pyautogui
import time

"""
446,214
656,385
1161,623
"""

pyautogui.moveTo(446,214)
time.sleep(0.5)
pyautogui.click(446,214)

pyautogui.moveTo(656,385)
time.sleep(0.5)
pyautogui.click(656,385)

pyautogui.moveTo(1161,623)
time.sleep(0.5)
pyautogui.click(1161,623)

